A Long Story to Tell
===
 經歷了一個學期的學習，希望各位同學從中會有所收穫，只要有運作，一定有數據，所以資料科學的重要性就在如何有效地 使用這些資料.就如同我們一直強調的，專業的認知才是最重要的根本，技術只是一個處理的 選擇，我們將你專業之外的知識及概念，在一個短短的學期內， 半強迫式地讓學生學習，同時為了你們將來的視野更廣泛，要求你們你們從團隊合作中，培養如何思考，協調，及合作，正如我們強調的工作不是一個人，而是團體的結果，希望你們能夠深切的體會.最後，我們就用 duckdb 示範如何探索 HuggingFace 的資料庫，作為 Ending!
 
 最後祝各位成為
 
    前端工程師 + 後端工程師  ➜ 全端工程師



In [ ]:
!pip install duckdb

In [1]:
import pandas 
import duckdb

In [2]:
# create an in-memory temporary database
## conn = duckdb.connect()
# or create embedded DBMS
conn = duckdb.connect('Score.duckdb')
c = conn.cursor()

In [ ]:
# Create tables by importing the content from the CSVs
score1='OnLineQuiz/math_2022_2.csv'
c.execute(
  "CREATE TABLE Math_2022_2 AS SELECT * FROM read_csv_auto(OnLineQuiz/math_2022_2.csv);"
).df()

In [3]:
score= c.execute('''
    SELECT 
        * 
    FROM read_csv_auto('OnLineQuiz/math_2022_2.csv')
''').df()

c.register("S2022_2", score)

In [4]:
c.sql("CREATE TABLE Math_2022_2 AS SELECT * FROM S2022_2")

In [5]:
c.sql('SHOW TABLES').df()

,name
0,Math_2022_2


In [ ]:
c.sql("select * from Math_2022_2")

In [7]:
c.sql("INSERT INTO Math_2022_2 (user,name) VALUES ('B0','B0')")

In [ ]:
c.sql("select * from Math_2022_2")

In [10]:
c.sql("UPDATE Math_2022_2 SET login=1,name='B1' WHERE user='B0';")

In [ ]:
c.sql("select * from Math_2022_2")

In [21]:
df = c.sql('SELECT * FROM Math_2022_2').df()
df[['user','login']]

,user,login
0,B0729004,NaN
1,B1003211,NaN
2,B1003213,NaN
3,B1003216,NaN
4,B1009242,NaN
5,B1009247,NaN
6,B1027152,NaN
7,B0,1.0


# HuggingFace DataBases

In [15]:
import requests

In [16]:
# Access the Datasets Server

r = requests.get("https://datasets-server.huggingface.co/parquet?dataset=blog_authorship_corpus")
j = r.json()
urls = [f['url'] for f in j['parquet_files'] if f['split'] == 'train']
urls
['https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00000-of-00002.parquet',
 'https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00001-of-00002.parquet']



['https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00000-of-00002.parquet',
 'https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00001-of-00002.parquet']

In [17]:
# Create a connection to DuckDB and install and load the httpfs extension to allow remote files accessing

url = "https://huggingface.co/datasets/blog_authorship_corpus/resolve/refs%2Fconvert%2Fparquet/blog_authorship_corpus/blog_authorship_corpus-train-00000-of-00002.parquet"

con = duckdb.connect()
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

In [18]:
# query example on the Database
# https://huggingface.co/docs/datasets-server/parquet_process

con.sql(f"""SELECT horoscope, 
    count(*), 
    AVG(LENGTH(text)) AS avg_blog_length 
    FROM '{url}' 
    GROUP BY horoscope 
    ORDER BY avg_blog_length 
    DESC LIMIT(5)
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────┬──────────────┬────────────────────┐
│ horoscope │ count_star() │  avg_blog_length   │
│  varchar  │    int64     │       double       │
├───────────┼──────────────┼────────────────────┤
│ Aquarius  │        34062 │  1129.218836239798 │
│ Cancer    │        41509 │  1098.366812016671 │
│ Capricorn │        33961 │ 1073.2002002296751 │
│ Libra     │        40302 │ 1072.0718326633914 │
│ Leo       │        40587 │ 1064.0536871412028 │
└───────────┴──────────────┴────────────────────┘